# Vertical Bin Packing Problem

- Airport and Cargo Operations (AE4446)
- Zach Kelly (5901405), Timotei Dudas ()
- May 5th, 2023

### Imports

In [10]:
import pandas as pd
import numpy as np 
import gurobipy as gp
from gurobipy import Model, GRB, GurobiError, quicksum

### Input Data

In [2]:
# load data 
ulds = pd.read_pickle('../data/B.pickle')
items = pd.read_pickle('../data/R.pickle')

print('----- ULDs -----')
display(ulds)
print('----- Items -----')
display(items)

----- ULDs -----


{0: (0, [300, 155, 2, 140, 0, 0, 0]),
 1: (0, [300, 155, 2, 140, 0, 0, 0]),
 2: (1, [192, 155, 2, 200, 0, 0, 0]),
 3: (1, [192, 155, 2, 200, 0, 0, 0])}

----- Items -----


{0: (98, 50, 1, 0, 0, 0),
 1: (89, 46, 1, 0, 0, 0),
 2: (64, 24, 1, 0, 0, 0),
 3: (93, 29, 1, 1, 0, 0),
 4: (114, 54, 1, 1, 0, 0),
 5: (95, 60, 1, 0, 0, 0),
 6: (63, 28, 1, 0, 0, 0),
 7: (100, 46, 0, 0, 0, 0),
 8: (52, 61, 1, 0, 0, 0),
 9: (45, 46, 1, 0, 0, 0),
 10: (111, 32, 1, 0, 0, 0),
 11: (109, 38, 1, 0, 1, 0),
 12: (97, 57, 1, 0, 0, 0),
 13: (51, 29, 1, 0, 0, 0),
 14: (86, 54, 1, 0, 0, 0),
 15: (114, 31, 1, 1, 0, 0),
 16: (81, 47, 1, 0, 0, 0),
 17: (78, 25, 1, 0, 0, 0),
 18: (78, 44, 1, 0, 0, 0),
 19: (68, 33, 1, 0, 0, 0),
 20: (51, 45, 1, 0, 0, 0),
 21: (84, 36, 1, 0, 0, 1),
 22: (66, 35, 0, 0, 0, 0),
 23: (68, 42, 1, 1, 0, 0),
 24: (108, 57, 1, 1, 1, 0)}

### Setup Model

In [3]:
print('Creating Model')
print('--------------------')
model = Model()

Creating Model
--------------------
Restricted license - for non-production use only - expires 2024-10-28


### Model Parameters 

In [9]:
# total number of boxes to be packed
n = len(items.keys())
# total number of available ULDs
m = len(ulds.keys())
# dimensions of items
li = [items[i][0] for i in range(n)] # length
hi = [items[i][1] for i in range(n)] # height
# dimensions of ULDs 
lc = [300,192]
hc = [155,155]
# maximum area of ULDs
Aj = [lc[j]*hc[j] for j in range(len(lc))]
# total area of items
Ai = np.sum([li[i]*hi[i] for i in range(len(li))])
# find largest uld length and width
Lj_max = np.max(np.array(list(dict(ulds.values()).values()))[:,0])
Hj_max = np.max(np.array(list(dict(ulds.values()).values()))[:,1])

### Model Variables

In [8]:
print('Creating Decision Variables')
print('--------------------')

### define decision variables ###

p_ij = {} # if item i is in ULD j 

for i in range(n):
    for j in range(m):
        p_ij[i,j]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="p[%s,%s]"%(i,j))

u_j = {} # if container j is used 
a_j = {} # unused area of container j 
for j in range(m):
    u_j[j] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="u[%s]"%(j))
    a_j[j] = model.addVar(lb=0, ub=Hj_max*Lj_max, vtype=GRB.CONTINUOUS, name="a[%s]"%(j))

x_i = {} # x location of lower left corner of item i
z_i = {} # z location of lower left corner of item i
x_i_ = {} # x location of upper right corner of item i
z_i_ = {} # z location of upper right corner of item i 

for i in range(n):
    x_i[i] = model.addVar(lb=0, ub=Lj_max, vtype=GRB.CONTINUOUS, name="x_[%s]"%(i))
    x_i_[i] = model.addVar(lb=0, ub=Lj_max, vtype=GRB.CONTINUOUS, name="x_[%s]_"%(i))
    z_i[i] = model.addVar(lb=0, ub=Hj_max, vtype=GRB.CONTINUOUS, name="z_[%s]"%(i))
    z_i_[i] = model.addVar(lb=0, ub=Hj_max, vtype=GRB.CONTINUOUS, name="z_[%s]_"%(i))

r_i11 = {} # if length is along x axis 
r_i13 = {} # if height is along x axis 
r_i31 = {} # if length is along z axis 
r_i33 = {} # if height is along z axis 

for i in range(n):
    r_i11[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="r_[%s]11"%(i))
    r_i13[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="r_[%s]13"%(i))
    r_i31[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="r_[%s]31"%(i))
    r_i33[i] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="r_[%s]33"%(i))

xp_ik = {} # if item i is to the right of item k 
zp_ik = {} # if item i is above item k 

for i in range(n):
    for k in range(i+1,n):
        xp_ik[1,k] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="xp_ik[%s,%s]"%(i,k))
        zp_ik[1,k] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="zp_ik[%s,%s]"%(i,k))

r_iab = {} # if side b of item i is along axis a 

for i in range(n):
    for a in [0,2]:
        for b in [0,2]:
            r_iab[i,a,b] = model.addVar(lb=0, ub=1, vtype=GRB.BINARY, name="r_iab[%s,%s,%s]"%(i,a,b))

model.update()     

Creating Decision Variables
--------------------


### Model Constraints

In [ ]:
print('Creating Constraints')
print('--------------------')

# item i is constrained to single uld j
for i in range(n):
    model.addConstr(quicksum(p_ij[i,j] for j in range(m) == 1))

# item dimensions must be less than or equal to the dimensions of the container
for j in range(m):
    model.addConstr(p_ij[i,j] * (x_i_[i] if r_i11[i] else z_i_ for i in range(n) <= Lj_max))
    model.addConstr(p_ij[i,j] * (z_i_[i] if r_i13[i] else x_i_ for i in range(n) <= Hj_max))


### Model Objective